### parse data for vcf

In [1]:
import pandas

In [3]:
def fix_transcript(row):
    return row['Transcript'].split('.')[0]

def choose_one(rows):
    if len(rows) == 1:
        return rows
    else:
        r = rows[rows.Classification != 'VUS']
        if len(r) == 1:
            return r
        else:
            return rows.iloc[0]

In [7]:
dat_file = '../data/raw/EPIv6.xlsx'
df_init = pandas.read_excel(dat_file)
crit = df_init.apply(lambda row: not pandas.isnull(row['Transcript']), axis=1)
df_pre = df_init[crit]
print( df_pre.head() )

# rm duplicate rows
df_pre.loc[:, 'simple_nm'] = df_pre.apply(fix_transcript, axis=1)
cols = [x for x in df_pre.columns.values
        if x != 'Transcript']
idx_vals = df_pre[cols].drop_duplicates().index.values
df_fix = df_pre.loc[idx_vals][df_init.columns.values]

# choose non-VUS row for duplicates
g_cols = [x for x in df_fix.columns.values
          if not x in ('Transcript', 'Classification')]
df_fix.groupby(g_cols).apply(choose_one)
df = df_fix[ df_fix['Ref (single-base)'] != 'no alleles found' ]
g_cols = ('chr', 'pos', 'ref', 'alt')
df.groupby(g_cols).size()

     chr     p.      c. Genomic Pos (hg19)  Ref (single-base)  \
0  chr22  A206S   c.616               12998                 G   
1  chr22  A260V   c.779               14480                 C   
2  chr22    A2V     c.5                 564                 C   
3  chr22  A480G  c.1439               20507                 C   
4  chr22  C483R  c.1447               20515                 T   

  Alt (single-base)  Ref  Alt Exon (Gene) Gene Symbol test/Indication  \
0                 T  GCC  TCC     exn5-39        ADSL        Comp_epi   
1                 T  GCA  GTA     exn7-14        ADSL        Comp_epi   
2                 T  GCG  GTG     exn1+64        ADSL      Infant_epi   
3                 G  GCA  GGA    exn13-68        ADSL      Infant_epi   
4                 C  TGT  CGT    exn13-60        ADSL       Child_epi   

    Transcript Classification  Pos Fam Cnt  Neg Fam Cnt  Homozygous Fam Cnt  \
0  NM_000026.2            VUS            8         8275                   0   
1  NM_000026

/opt/conda/lib/python3.4/site-packages/pandas/core/indexing.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/lib/python3.4/site-packages/pandas/core/indexing.py:477: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/opt/conda/lib/python3.4/site-packages/pandas/indexes/api.py:71: RuntimeWarning: unorderable types: int() < str(), sort order is undefined for incomparable objects
  result = result.union(other)
/opt/conda/lib/python3.4/site-packages/pandas/indexes/api.py:71

KeyError: 'chrom'

In [40]:
def fix_ref(row):
    ref = row['Ref (single-base)']
    if ',' in ref:
        return ref.split(',')[-1].split('>')[0]
    return ref

def fix_alt(row):
    ref = row['Ref (single-base)']
    if ',' in ref:
        return ref.split(',')[-1].split('>')[1]
    return row['Alt (single-base)']

def mk_vcf_line(row, fout):
    ls = (row['clinical_class'], row['pos_fam_count'], row['neg_fam_count'])
    info = 'CLIN_CLASS=%s;POS_FAM_COUNT=%d;NEG_FAM_COUNT=%d' % ls
    ls  = (row['chrom'], str(row['pos']), '.',
           row['ref'], row['alt'], '.', '.',
           info)
    print('\t'.join(ls), file=fout)

def write_vcf(df, vcf_out):
    with open(vcf_out, 'w') as fout:
        print('##fileformat=VCFv4.2', file=fout)
        print('##INFO=<ID=CLIN_CLASS,Number=1,Type=String,Description="pos_fam_count">', file=fout)
        print('##INFO=<ID=POS_FAM_COUNT,Number=1,Type=Integer,Description="pos_fam_count">', file=fout)
        print('##INFO=<ID=NEG_FAM_COUNT,Number=1,Type=Integer,Description="pos_fam_count">', file=fout)
        print('#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO', file=fout)        
        df.apply(lambda row: mk_vcf_line(row, fout), axis=1)
        
df.loc[:, 'ref'] = df.apply(fix_ref, axis=1)
df.loc[:, 'alt'] = df.apply(fix_alt, axis=1)
df.loc[:, 'chrom'] = df.apply(lambda row: row['chr'][3:], axis=1)
uc = cols = ['chrom', 'Genomic Pos (hg19) ', 'ref', 'alt',
             'Classification', 'Pos Fam Cnt', 'Neg Fam Cnt', 'Homozygous Fam Cnt',
             'Hemizygous Fam Cnt', 'Heterozygous Fam Cnt']
new_cols = {'Genomic Pos (hg19) ': 'pos',
            'Classification':'clinical_class',
            'Pos Fam Cnt':'pos_fam_count',
            'Neg Fam Cnt':'neg_fam_count',
            'Homozygous Fam Cnt':'hom_fam_count',
            'Hemizygous Fam Cnt':'hemi_fam_count',
            'Heterozygous Fam Cnt':'het_fam_count'
           }
df_final = df[uc].rename(columns=new_cols).sort_values(by=['chrom', 'pos'])
vcf_out = '../data/interim/EPIv6.vcf'
write_vcf(df_final, vcf_out)

/Users/perry/miniconda3/envs/mahdi_epi/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [26]:
df_final.head()

,chrom,pos,ref,alt,clinical_class,pos_fam_count,neg_fam_count,hom_fam_count,hemi_fam_count,het_fam_count
0,22,12998,G,G,VUS,8,8275,0,0,8
1,22,14480,C,C,VUS,1,8282,0,0,1
2,22,564,C,C,PATHOGENIC,1,8282,0,0,1
3,22,20507,C,C,VUS,1,8282,0,0,1
4,22,20515,T,T,VUS,1,8282,0,0,1


In [37]:
df[ df['Genomic Pos (hg19) ']==1435]

,chr,p.,c.,Genomic Pos (hg19),Ref (single-base),Alt (single-base),Ref,Alt,Exon (Gene),Gene Symbol,...,Transcript,Classification,Pos Fam Cnt,Neg Fam Cnt,Homozygous Fam Cnt,Hemizygous Fam Cnt,Heterozygous Fam Cnt,ref,alt,chrom
2542,chr13,M1?,c.2,1435,no alleles found,NaN,NNN,NNN,exn1+30,CLN5,...,NM_006493.2,PATHOGENIC,10,8535,0,0,10,no alleles found,NaN,13
4033,chr13,Met1?,c.2,1435,T,C,T,C,NaN,CLN5,...,NaN,VUS,1,8544,0,0,1,T,C,13


In [21]:
crit = df.apply(lambda row: ',' in row['Ref (single-base)'], axis=1)
uc = ['p.', 'Ref (single-base)', 'Alt (single-base)', 'ref', 'alt']
df[crit][uc]

,p.,Ref (single-base),Alt (single-base),ref,alt
221,NaN,"multiple alleles found: G>A,G>T,G>C",NaN,G,C
297,NaN,"multiple alleles found: C>G,C>T",NaN,C,T
298,S1282_G1285del,"multiple alleles found: CTGCCAAGGA>G,CTGCCAAGG...",NaN,CTGCCAAGG,G
299,S1282_G1285delinsR,"multiple alleles found: CTGCCAAGGA>G,CTGCCAAGG...",NaN,CTGCCAAGG,G
300,NaN,"multiple alleles found: G>C,G>T",NaN,G,T
1026,NaN,"multiple alleles found: C>A,C>T",NaN,C,T
1256,H1003LfsX29,"multiple alleles found: A>-,A>G",NaN,A,G
1952,Q52_Q55del,"multiple alleles found: ->GCAGCAGCAGCA,GCAGCAG...",NaN,GCAGCAGCAGCA,-
1953,Q51_Q55del,"multiple alleles found: ->GCAGCAGCAGCAGCA,GCAG...",NaN,GCAGCAGCAGCAGCA,-
2392,Q70_P78dup,multiple alleles found: AGCAGCAGCAGCCGCCGCCGCC...,NaN,-,AGCAGCAGCAGCCGCCGCCGCCGCCGC


In [15]:
df.head()

,chr,p.,c.,Genomic Pos (hg19),Ref (single-base),Alt (single-base),Ref,Alt,Exon (Gene),Gene Symbol,test/Indication,Transcript,Classification,Pos Fam Cnt,Neg Fam Cnt,Homozygous Fam Cnt,Hemizygous Fam Cnt,Heterozygous Fam Cnt,ref
0,chr22,A206S,c.616,12998,G,T,GCC,TCC,exn5-39,ADSL,Comp_epi,NM_000026.2,VUS,8,8275,0,0,8,G
1,chr22,A260V,c.779,14480,C,T,GCA,GTA,exn7-14,ADSL,Comp_epi,NM_000026.2,VUS,1,8282,0,0,1,C
2,chr22,A2V,c.5,564,C,T,GCG,GTG,exn1+64,ADSL,Infant_epi,NM_000026.2,PATHOGENIC,1,8282,0,0,1,C
3,chr22,A480G,c.1439,20507,C,G,GCA,GGA,exn13-68,ADSL,Infant_epi,NM_000026.2,VUS,1,8282,0,0,1,C
4,chr22,C483R,c.1447,20515,T,C,TGT,CGT,exn13-60,ADSL,Child_epi,NM_000026.2,VUS,1,8282,0,0,1,T


In [17]:
df.iloc[1953]

chr                                                                 chr15
p.                                                             Q51_Q55del
c.                                                              c.144_158
Genomic Pos (hg19)                                                   1685
Ref (single-base)       multiple alleles found: ->GCAGCAGCAGCAGCA,GCAG...
Alt (single-base)                                                     NaN
Ref                                                                   NaN
Alt                                                                   NaN
Exon (Gene)                                                           NaN
Gene Symbol                                                          POLG
test/Indication                                                  Comp_epi
Transcript                                                    NM_002693.2
Classification                                              LIKELY BENIGN
Pos Fam Cnt                           